In [13]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)
print(latest_file)

graph_list/Town01\1_graph_v3.adjlist


In [16]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end = (547, 1391), (1595, 539)

if reverse == "y":
    start, end = end, start


In [17]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "Forward", True)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (547, 1391)
End point: (1595, 539)
Finding Path...
Get!!!
found!!!
[(547, 1391), (547, 1386), (547, 1383), (547, 1382), (547, 1379), (547, 1378), (547, 1375), (547, 1374), (547, 1371), (547, 1370), (547, 1367), (547, 1366), (547, 1363), (547, 1362), (547, 1359), (547, 1358), (547, 1355), (547, 1354), (547, 1350), (547, 1346), (547, 1342), (547, 1338), (547, 1334), (547, 1330), (547, 1326), (547, 1322), (547, 1318), (547, 1314), (547, 1310), (547, 1306), (547, 1302), (547, 1298), (547, 1294), (547, 1290), (547, 1286), (547, 1282), (547, 1278), (547, 1274), (547, 1270), (547, 1266), (547, 1262), (547, 1258), (547, 1254), (547, 1250), (547, 1246), (547, 1242), (547, 1238), (547, 1234), (547, 1230), (547, 1226), (547, 1222), (547, 1218), (547, 1214), (547, 1210), (547, 1206), (547, 1202), (547, 1198), (547, 1194), (547, 1190), (547, 1186), (547, 1182), (547, 1178), (547, 1174), (547, 1170), (547, 1166), (547, 1162), (547, 1158), (547, 1154), (547, 1150), (547, 1146), (547, 114

In [18]:
############### Quick test for other start point with the same goal point
start, end = (547, 1391), (3873, 1344)

fp = nx.shortest_path(G,  start,  end)
print(fp)

[(547, 1391), (547, 1386), (547, 1382), (547, 1378), (547, 1374), (547, 1370), (547, 1366), (547, 1362), (547, 1358), (547, 1354), (547, 1350), (547, 1342), (547, 1338), (547, 1330), (547, 1326), (547, 1318), (547, 1314), (547, 1306), (547, 1302), (547, 1294), (547, 1290), (547, 1282), (547, 1278), (547, 1270), (547, 1266), (547, 1258), (547, 1254), (547, 1246), (547, 1242), (547, 1234), (547, 1230), (547, 1222), (547, 1218), (547, 1210), (547, 1206), (547, 1198), (547, 1194), (547, 1186), (547, 1182), (547, 1174), (547, 1170), (547, 1162), (547, 1158), (547, 1150), (547, 1146), (547, 1138), (547, 1134), (547, 1126), (547, 1122), (547, 1114), (547, 1110), (547, 1102), (547, 1098), (547, 1090), (547, 1086), (547, 1078), (547, 1074), (547, 1066), (547, 1062), (547, 1054), (547, 1050), (547, 1042), (547, 1038), (547, 1030), (547, 1026), (547, 1018), (547, 1014), (547, 1006), (547, 1002), (547, 994), (547, 990), (547, 982), (547, 978), (547, 970), (547, 966), (547, 958), (547, 954), (547, 